In [3]:
## imports
import pandas as pd
import numpy as np
import re
import requests
#import yaml


## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 1. Example 1: no credentials; no wrapper

Site: National Assessment of Education Progress (NAEP)

Documentation: https://www.nationsreportcard.gov/api_documentation.aspx

Base link: https://www.nationsreportcard.gov/DataService/GetAdhocData.aspx 

## 1.1 Query to pull some data

In [4]:
## using their example query of 2011 writing scores separated by gender
## based on here - https://stackoverflow.com/questions/40836749/pythonic-way-of-writing-a-single-line-long-string
## using the ( ) syntax to formulate a long
## string without linebreaks added
example_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011')


example_naep_query


'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'

In [5]:
## use requests to call the api
naep_resp = requests.get(example_naep_query)
naep_resp
print(type(naep_resp))

## get the json contents of the response 
## here, we're assuming valid response
naep_resp_j = naep_resp.json()
naep_resp_j

## with result, turn it into a dataframe
naep_resp_d = pd.DataFrame(naep_resp_j['result'])
naep_resp_d

<Response [200]>

<class 'requests.models.Response'>


{'status': 200,
 'result': [{'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '1',
   'varValueLabel': 'Male',
   'value': 139.099504632971,
   'isStatDisplayable': 1,
   'errorFlag': 0},
  {'year': 2011,
   'sample': 'R3',
   'yearSampleLabel': '2011',
   'Cohort': 2,
   'CohortLabel': 'Grade 8',
   'stattype': 'MN:MN',
   'subject': 'WRI',
   'grade': 8,
   'scale': 'WRIRP',
   'jurisdiction': 'NP',
   'jurisLabel': 'National public',
   'variable': 'GENDER',
   'variableLabel': 'Gender',
   'varValue': '2',
   'varValueLabel': 'Female',
   'value': 158.567104984955,
   'isStatDisplayable': 1,
   'errorFlag': 0}]}

,year,sample,yearSampleLabel,Cohort,CohortLabel,stattype,subject,grade,scale,jurisdiction,jurisLabel,variable,variableLabel,varValue,varValueLabel,value,isStatDisplayable,errorFlag
0,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,1,Male,139.099505,1,0
1,2011,R3,2011,2,Grade 8,MN:MN,WRI,8,WRIRP,NP,National public,GENDER,Gender,2,Female,158.567105,1,0


## 1.2 What happens if there's an error in our query?

In [6]:
## here's a query that from the documentation we know
## won't work since i modified year to 2025 which doesnt
## exist in the data
wrong_naep_query = (
'https://www.nationsreportcard.gov/'
'Dataservice/GetAdhocData.aspx?'
'type=data&subject=writing&grade=8&'
'subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025')

wrong_naep_query

'https://www.nationsreportcard.gov/Dataservice/GetAdhocData.aspx?type=data&subject=writing&grade=8&subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2025'

In [7]:
## use requests to call the api
naep_wrong_resp = requests.get(wrong_naep_query)
naep_wrong_resp

<Response [200]>

In [8]:
## in the case of this particular api,
## the call returns some response but
## when we try to extract the json containing
## status or results, we get in an error
#naep_wrong_resp.json() # uncomment to see error

### 1.2.2 More all-purpose way of allowing remainder of calls to run: try, except

In [9]:
## putting it in a try; except as general error catching
try:
    results = naep_wrong_resp.json()['result']
except Exception as e:
    print('Failed to get result from API due to error:')
    print(e) # or just: pass

Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)


### 1.2.3 Can usually also find more targeted way but that varies more across APIs

In [10]:
## if we wanted do more specific error catching,
## see that the status == 400 actually appears here
## so could write if else along those lines
naep_wrong_resp.text
naep_resp.text

if "System.Exception" in naep_wrong_resp.text:
    print("NAEP results not found")

'{"status":400,"result": "System.Exception: The query \'SELECT DISTINCT Framework FROM Cycles WHERE Subject=\'WRI\' AND Cohort=2 AND CONVERT(VARCHAR(10),Year)+Sample IN (\'2025R3\')\' did not return exactly 1 framework. Make sure you can trend the years defined for the given subject and cohort.\r\n   at NRCDataService3.GetAdhocData.GetFramework(NDEContext& ndeContext, String subjectCode, List`1 yearSamples, String cohort) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 2091\r\n   at NRCDataService3.GetAdhocData.PopulateBaseOrchestratorRequest() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 1781\r\n   at NRCDataService3.GetAdhocData.ConstructRequest_Datapoint() in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 628\r\n   at NRCDataService3.GetAdhocData.Page_Load(Object sender, EventArgs e) in C:\\projects\\ndecore2015\\NRCDataService2\\GetAdhocData.aspx.cs:line 179"}'

'{"status":200,"result": [{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"1","varValueLabel":"Male","value":139.099504632971,"isStatDisplayable":1,"errorFlag":0},{"year":2011,"sample":"R3","yearSampleLabel":"2011","Cohort":2,"CohortLabel":"Grade 8","stattype":"MN:MN","subject":"WRI","grade":8,"scale":"WRIRP","jurisdiction":"NP","jurisLabel":"National public","variable":"GENDER","variableLabel":"Gender","varValue":"2","varValueLabel":"Female","value":158.567104984955,"isStatDisplayable":1,"errorFlag":0}]}'

NAEP results not found


## Activity 1: writing a function to make multiple, sequential calls

- Say we want to pull the data for grades 4, 8, and 12
- How can we write a function that iterates over a list of those grades and pulls the data for each grade?

**Note**: an ideal function would have arguments for each parameter in the API like subject, subscale, etc. Here we can leave those other parts constant

In [11]:
# your code here
def doGrade(grade):
    link = 'https://www.nationsreportcard.gov/''Dataservice/GetAdhocData.aspx?''type=data&subject=writing&grade=' + str(grade) + '&''subscale=WRIRP&variable=GENDER&jurisdiction=NP&stattype=MN:MN&Year=2011'
    naep_resp1 = requests.get(link)
    print(naep_resp1)
    print(type(naep_resp1))
    try:
        results = naep_resp1.json()['result']
        return results
    except Exception as e:
        print('Failed to get result from API due to error:')
        print(e) # or just: pass
doGrade(4)
doGrade(8)
doGrade(12)

<Response [200]>
<class 'requests.models.Response'>
Failed to get result from API due to error:
Invalid control character at: line 1 column 289 (char 288)
<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 139.099504632971,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 2,
  'CohortLabel': 'Grade 8',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 8,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 158.567104984955,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

<Response [200]>
<class 'requests.models.Response'>


[{'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '1',
  'varValueLabel': 'Male',
  'value': 141.256977963264,
  'isStatDisplayable': 1,
  'errorFlag': 0},
 {'year': 2011,
  'sample': 'R3',
  'yearSampleLabel': '2011',
  'Cohort': 3,
  'CohortLabel': 'Grade 12',
  'stattype': 'MN:MN',
  'subject': 'WRI',
  'grade': 12,
  'scale': 'WRIRP',
  'jurisdiction': 'NP',
  'jurisLabel': 'National public',
  'variable': 'GENDER',
  'variableLabel': 'Gender',
  'varValue': '2',
  'varValueLabel': 'Female',
  'value': 155.385916780351,
  'isStatDisplayable': 1,
  'errorFlag': 0}]

# 2. Example 2: needs credentials; no wrapper

Create an account here: https://www.yelp.com/developers/v3/manage_app

In [12]:
## get the key
API_KEY = "m6B2Dw4PE9ckx55EtA9udlpXUmuYv5L0BHPk63NJc4_YSiYovfnJiApIYPZTfMbFGHuetjEUdVwiqGauXEal5Q1Rp_Sr9txshHjiE-lGMClmlpcKoVMiQbHKR60zZnYx"

In [16]:
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Hanover,NH,03755"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()


<Response [200]>

In [17]:
## example business
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

{'id': 'XVGEEIH5rVB2QzW-qywcJw',
 'alias': 'base-camp-cafe-hanover',
 'name': 'Base Camp Cafe',
 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiEQI7p_JF82eaWa9Xg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/base-camp-cafe-hanover?adjust_creative=sWa2Cp4ZwbZGeXh93Xt1zw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sWa2Cp4ZwbZGeXh93Xt1zw',
 'review_count': 247,
 'categories': [{'alias': 'himalayan', 'title': 'Himalayan/Nepalese'}],
 'rating': 4.4,
 'coordinates': {'latitude': 43.700626, 'longitude': -72.2887803},
 'transactions': ['delivery'],
 'price': '$$',
 'location': {'address1': '3 Lebanon St',
  'address2': 'Ste 13',
  'address3': '',
  'city': 'Hanover',
  'zip_code': '03755',
  'country': 'US',
  'state': 'NH',
  'display_address': ['3 Lebanon St', 'Ste 13', 'Hanover, NH 03755']},
 'phone': '+16036432007',
 'display_phone': '(603) 643-2007',
 'distance': 196.1397581876442,
 'attributes': {'business_temp_closed': None,
  'menu

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance,attributes
0,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,False,https://www.yelp.com/biz/base-camp-cafe-hanove...,247,"[{'alias': 'himalayan', 'title': 'Himalayan/Ne...",4.4,"{'latitude': 43.700626, 'longitude': -72.2887803}",[delivery],$$,"{'address1': '3 Lebanon St', 'address2': 'Ste ...",+16036432007,(603) 643-2007,196.139758,"{'business_temp_closed': None, 'menu_url': 'ht..."
1,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,False,https://www.yelp.com/biz/mollys-restaurant-and...,508,"[{'alias': 'tradamerican', 'title': 'American'...",3.9,"{'latitude': 43.701144, 'longitude': -72.2894249}",[delivery],$$,"{'address1': '43 South Main St', 'address2': '...",+16036432570,(603) 643-2570,250.830160,"{'business_temp_closed': None, 'menu_url': 'ht..."
2,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,False,https://www.yelp.com/biz/sawtooth-kitchen-hano...,25,"[{'alias': 'chickenshop', 'title': 'Chicken Sh...",4.1,"{'latitude': 43.70158, 'longitude': -72.289641}",[],NaN,"{'address1': '33 S Main St', 'address2': '', '...",+16036435134,(603) 643-5134,242.607552,"{'business_temp_closed': None, 'menu_url': 'ht..."
3,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,False,https://www.yelp.com/biz/duende-hanover-2?adju...,11,"[{'alias': 'spanish', 'title': 'Spanish'}, {'a...",3.8,"{'latitude': 43.70074272457673, 'longitude': -...",[],NaN,"{'address1': '15 Lebanon St', 'address2': '', ...",+16033064826,(603) 306-4826,107.388862,"{'business_temp_closed': None, 'menu_url': 'ht..."
4,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,False,https://www.yelp.com/biz/lous-restaurant-and-b...,383,"[{'alias': 'tradamerican', 'title': 'American'...",4.2,"{'latitude': 43.70143, 'longitude': -72.289001}",[delivery],$$,"{'address1': '30 S Main St', 'address2': '', '...",+16036433321,(603) 643-3321,244.006059,"{'business_temp_closed': None, 'menu_url': 'ht..."


In [18]:
## more data-specific way of summarizing
## we're doing a simple approach and just retaining
## cols that have a simple str structure
## if doing for real, would want to extract things
def clean_yelp_json(one_biz):

    ## restrict to str cols
    d_str = {key:value for key, value in one_biz.items()
             if type(value) == str}
    
    df_str = pd.DataFrame(d_str, index = [d_str['id']])
    return(df_str)

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


,id,alias,name,image_url,url,price,phone,display_phone
XVGEEIH5rVB2QzW-qywcJw,XVGEEIH5rVB2QzW-qywcJw,base-camp-cafe-hanover,Base Camp Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/p8_YiE...,https://www.yelp.com/biz/base-camp-cafe-hanove...,$$,+16036432007,(603) 643-2007
8ybF6YyRldtZmU9jil4xlg,8ybF6YyRldtZmU9jil4xlg,mollys-restaurant-and-bar-hanover,Molly's Restaurant & Bar,https://s3-media4.fl.yelpcdn.com/bphoto/TJLrrA...,https://www.yelp.com/biz/mollys-restaurant-and...,$$,+16036432570,(603) 643-2570
5WW4g_LRwau29KyjZGLyAA,5WW4g_LRwau29KyjZGLyAA,sawtooth-kitchen-hanover,Sawtooth Kitchen,https://s3-media3.fl.yelpcdn.com/bphoto/61MNG4...,https://www.yelp.com/biz/sawtooth-kitchen-hano...,NaN,+16036435134,(603) 643-5134
qxm3VNmD0O2zw8m9U8SxZg,qxm3VNmD0O2zw8m9U8SxZg,duende-hanover-2,Duende,https://s3-media4.fl.yelpcdn.com/bphoto/rf_h2D...,https://www.yelp.com/biz/duende-hanover-2?adju...,NaN,+16033064826,(603) 306-4826
KA8yhrd-ClVYMyOefXdVYg,KA8yhrd-ClVYMyOefXdVYg,lous-restaurant-and-bakery-hanover,Lou's Restaurant & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/VAx8H9...,https://www.yelp.com/biz/lous-restaurant-and-b...,$$,+16036433321,(603) 643-3321
jAFLKiVXq9z_h4ZPy0nLMA,jAFLKiVXq9z_h4ZPy0nLMA,the-works-café-hanover,The Works Café,https://s3-media2.fl.yelpcdn.com/bphoto/olpsy-...,https://www.yelp.com/biz/the-works-caf%C3%A9-h...,NaN,+16032772082,(603) 277-2082
vMyN7JL5cJExJORgIobbQg,vMyN7JL5cJExJORgIobbQg,tuk-tuk-thai-cuisine-hanover,Tuk Tuk Thai Cuisine,https://s3-media4.fl.yelpcdn.com/bphoto/--bUG3...,https://www.yelp.com/biz/tuk-tuk-thai-cuisine-...,$$,+16032779192,(603) 277-9192


# Activity 2: pull restaurants in a different location

- Try running a business search query for your hometown or another place by constructing a query similar to `yelp_genquery` but changing the location parameter
- Other endpoints require feeding what's called the business' fusion id into the API. Take an id from `yelp_stronly.id` and use the documentation here to pull the reviews for that business: https://docs.developer.yelp.com/reference/v3_business_reviews
- **Challenge**: generalize the previous step by writing a function that (1) takes a list of business ids as an input, (2) calls the reviews API for each id, (3) returns the results, and (4) rowbinds all results, i.e. turns them into a single, usable DataFrame

In [22]:
# your code here
## use documentation to define what to search
## doc: https://www.yelp.com/developers/documentation/v3/business_search
## write the query 
base_url = "https://api.yelp.com/v3/businesses/search?"
my_name = "restaurants"
my_location = "Lexington,MA,02420"
yelp_genquery = ('{base_url}'
                'term={name}'
                '&location={loc}').format(base_url = base_url,
                name = my_name,
                loc = my_location)

## use requests to call the API; here, we're
## passing it our credentials (structure varies
## by API and telling it to only return 10 results
## (max is 50 at once)
header = {'Authorization': f"Bearer {API_KEY}"}
yelp_genresp = requests.get(yelp_genquery, headers = header)
yelp_genresp

## then, look at structure of response
yelp_genjson = yelp_genresp.json()
yelp_genjson['businesses'][0]

## more automatic way of summarizing but things end up in lists
## within columns for things like categories
yelp_gendf = pd.DataFrame(yelp_genjson['businesses'])
yelp_gendf.head()

yelp_stronly = [clean_yelp_json(one_b) for one_b in yelp_genjson['businesses']]
yelp_stronly_df = pd.concat(yelp_stronly)

yelp_stronly_df.head(7)


<Response [200]>

{'id': '2oaZx5hc7DhPvqdJjwPGsQ',
 'alias': 'the-fox-den-woburn',
 'name': 'The Fox Den',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/QRzzluIzgwnBQKSOjrlIfg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/the-fox-den-woburn?adjust_creative=sWa2Cp4ZwbZGeXh93Xt1zw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sWa2Cp4ZwbZGeXh93Xt1zw',
 'review_count': 48,
 'categories': [{'alias': 'polish', 'title': 'Polish'},
  {'alias': 'portuguese', 'title': 'Portuguese'},
  {'alias': 'cocktailbars', 'title': 'Cocktail Bars'}],
 'rating': 4.4,
 'coordinates': {'latitude': 42.48191471896275,
  'longitude': -71.15077732175573},
 'transactions': ['pickup', 'delivery'],
 'location': {'address1': '482 Main St',
  'address2': '',
  'address3': None,
  'city': 'Woburn',
  'zip_code': '01801',
  'country': 'US',
  'state': 'MA',
  'display_address': ['482 Main St', 'Woburn, MA 01801']},
 'phone': '+17813053869',
 'display_phone': '(781) 305-3869',
 'distance': 6

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,attributes,price
0,2oaZx5hc7DhPvqdJjwPGsQ,the-fox-den-woburn,The Fox Den,https://s3-media2.fl.yelpcdn.com/bphoto/QRzzlu...,False,https://www.yelp.com/biz/the-fox-den-woburn?ad...,48,"[{'alias': 'polish', 'title': 'Polish'}, {'ali...",4.4,"{'latitude': 42.48191471896275, 'longitude': -...","[pickup, delivery]","{'address1': '482 Main St', 'address2': '', 'a...",+17813053869,(781) 305-3869,6063.732965,"{'business_temp_closed': None, 'menu_url': 'ht...",NaN
1,Un1kZZf5_TkxppTNkckc4Q,love-at-first-bite-lexington,Love at First Bite,https://s3-media4.fl.yelpcdn.com/bphoto/zG1Mji...,False,https://www.yelp.com/biz/love-at-first-bite-le...,200,"[{'alias': 'thai', 'title': 'Thai'}, {'alias':...",3.9,"{'latitude': 42.4468418, 'longitude': -71.226387}",[delivery],"{'address1': '1710 Massachusetts Ave', 'addres...",+17818625888,(781) 862-5888,1422.408970,"{'business_temp_closed': None, 'menu_url': Non...",$$
2,bDVyYogau6V435T4XtHkBw,my-other-kitchen-belmont,My Other Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/A5pNtW...,False,https://www.yelp.com/biz/my-other-kitchen-belm...,340,"[{'alias': 'greek', 'title': 'Greek'}, {'alias...",4.6,"{'latitude': 42.39300527705681, 'longitude': -...","[pickup, delivery]","{'address1': '762 Pleasant St', 'address2': ''...",+16179321444,(617) 932-1444,7709.340185,"{'business_temp_closed': None, 'menu_url': Non...",$$
3,VsU9PZWnKzAwfj-xpZl8rA,kin-dee-thai-and-pho-lexington,Kin Dee Thai & Pho,https://s3-media4.fl.yelpcdn.com/bphoto/VF6B6l...,False,https://www.yelp.com/biz/kin-dee-thai-and-pho-...,50,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 42.4587953835909, 'longitude': -7...","[pickup, delivery]","{'address1': '182 Bedford St', 'address2': '',...",+17818323939,(781) 832-3939,1759.426315,"{'business_temp_closed': None, 'menu_url': 'ht...",NaN
4,GoSabVesShIYhktvZjYV5w,burtons-grill-and-bar-burlington,Burtons Grill & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/dSvEDB...,False,https://www.yelp.com/biz/burtons-grill-and-bar...,451,"[{'alias': 'newamerican', 'title': 'New Americ...",3.8,"{'latitude': 42.46891, 'longitude': -71.20841}","[pickup, delivery]","{'address1': '43 Middlesex Turnpike', 'address...",+17812212281,(781) 221-2281,1454.034262,"{'business_temp_closed': None, 'menu_url': 'ht...",$$


,id,alias,name,image_url,url,phone,display_phone,price
2oaZx5hc7DhPvqdJjwPGsQ,2oaZx5hc7DhPvqdJjwPGsQ,the-fox-den-woburn,The Fox Den,https://s3-media2.fl.yelpcdn.com/bphoto/QRzzlu...,https://www.yelp.com/biz/the-fox-den-woburn?ad...,+17813053869,(781) 305-3869,NaN
Un1kZZf5_TkxppTNkckc4Q,Un1kZZf5_TkxppTNkckc4Q,love-at-first-bite-lexington,Love at First Bite,https://s3-media4.fl.yelpcdn.com/bphoto/zG1Mji...,https://www.yelp.com/biz/love-at-first-bite-le...,+17818625888,(781) 862-5888,$$
bDVyYogau6V435T4XtHkBw,bDVyYogau6V435T4XtHkBw,my-other-kitchen-belmont,My Other Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/A5pNtW...,https://www.yelp.com/biz/my-other-kitchen-belm...,+16179321444,(617) 932-1444,$$
VsU9PZWnKzAwfj-xpZl8rA,VsU9PZWnKzAwfj-xpZl8rA,kin-dee-thai-and-pho-lexington,Kin Dee Thai & Pho,https://s3-media4.fl.yelpcdn.com/bphoto/VF6B6l...,https://www.yelp.com/biz/kin-dee-thai-and-pho-...,+17818323939,(781) 832-3939,NaN
GoSabVesShIYhktvZjYV5w,GoSabVesShIYhktvZjYV5w,burtons-grill-and-bar-burlington,Burtons Grill & Bar,https://s3-media2.fl.yelpcdn.com/bphoto/dSvEDB...,https://www.yelp.com/biz/burtons-grill-and-bar...,+17812212281,(781) 221-2281,$$
NX19BZuBNK-lZey8NoH0bA,NX19BZuBNK-lZey8NoH0bA,the-bancroft-burlington-2,The Bancroft,https://s3-media3.fl.yelpcdn.com/bphoto/EUeyqk...,https://www.yelp.com/biz/the-bancroft-burlingt...,+17812212100,(781) 221-2100,$$$
uY9LCyN6GBLNGaF5mfEuug,uY9LCyN6GBLNGaF5mfEuug,town-meeting-bistro-lexington,Town Meeting Bistro,https://s3-media2.fl.yelpcdn.com/bphoto/996S1H...,https://www.yelp.com/biz/town-meeting-bistro-l...,+17813016655,(781) 301-6655,$$$


In [25]:
yelp_stronly_df.id.head()

2oaZx5hc7DhPvqdJjwPGsQ    2oaZx5hc7DhPvqdJjwPGsQ
Un1kZZf5_TkxppTNkckc4Q    Un1kZZf5_TkxppTNkckc4Q
bDVyYogau6V435T4XtHkBw    bDVyYogau6V435T4XtHkBw
VsU9PZWnKzAwfj-xpZl8rA    VsU9PZWnKzAwfj-xpZl8rA
GoSabVesShIYhktvZjYV5w    GoSabVesShIYhktvZjYV5w
Name: id, dtype: object

In [33]:
base_url = "https://api.yelp.com/v3/businesses/{}/reviews?"
header = {'Authorization': f"Bearer {API_KEY}"}

def get_business_reviews(business_ids):
    total_reviews = []
    for bus_id in business_ids:
        total_query = base_url.format(bus_id)
        yelp_genresp = requests.get(total_query, headers = header)
        yelp_genresp
        yelp_genjson = yelp_genresp.json()['reviews']
        yelp_genjson
        for rev_dict in yelp_genjson:
            rev_dict["business_id"] = bus_id
        total_reviews += yelp_genjson
    final_df = pd.DataFrame(total_reviews)
    return final_df
get_business_reviews(["Un1kZZf5_TkxppTNkckc4Q", "VsU9PZWnKzAwfj-xpZl8rA", "GoSabVesShIYhktvZjYV5w"])



KeyError: 'reviews'

In [26]:
one_biz = yelp_stronly_df

{"error": {"code": "VALIDATION_ERROR", "description": "'m6B2Dw4PE9ckx55EtA9udlpXUmuYv5L0BHPk63NJc4_YSiYovfnJiApIYPZTfMbFGHuetjEUdVwiqGauXEal5Q1Rp_Sr9txshHjiE-lGMClmlpcKoVMiQbHKR60zZnYx' does not match '^(?i)Bearer [A-Za-z0-9\\\\-\\\\_]{128}$'", "field": "Authorization", "instance": "m6B2Dw4PE9ckx55EtA9udlpXUmuYv5L0BHPk63NJc4_YSiYovfnJiApIYPZTfMbFGHuetjEUdVwiqGauXEal5Q1Rp_Sr9txshHjiE-lGMClmlpcKoVMiQbHKR60zZnYx"}}
